# Relatório 2

**Nome:** Thiago Lopes <br>
**Matrícula:** 20100358 <br>
**Turma:** T2

# Bibliotecas python utilizadas

In [122]:
import math
import matplotlib.pyplot as plt
import numpy as np
from prettytable import PrettyTable
import sympy


# Funçoes Auxiliares

In [123]:
def pivot(A, b, k):
    n = len(b)
    
    max_index = np.argmax(np.abs(A[k:n, k])) + k
    
    if max_index != k:
        A[[k, max_index]] = A[[max_index, k]]
        b[[k, max_index]] = b[[max_index, k]]

In [124]:
def back_substitution(U, y):
    m = len(y)
    x = np.zeros(m)
    
    for i in range(m - 1, -1, -1):
        x[i] = round((y[i] - np.dot(U[i, i + 1:], x[i + 1:])) / U[i, i], 3)
        
    return x


In [125]:
def forward_substitution(L, b):
    n = L.shape[0]
    y = np.zeros_like(b)
    
    for i in range(n):
        y[i] = (b[i] - np.dot(L[i, :i], y[:i])) / L[i, i]
    
    return y


In [126]:
def forward_substitution_LU(L, b, Pivot):
    m = len(b)
    y = np.zeros(m)
    
    for i in range(m):
        y[i] = b[Pivot[i] - 1] - np.dot(L[i, :i], y[:i])        
    return y


# Métodos implementados: Gaussian Elimination, LU Decomposition, Cholesky Decomposition,:

In [127]:
def gauss_elimination(A, b, use_pivoting=False):
    n = len(b)
    A = A.astype(float)
    b = b.astype(float)

    for k in range(n-1):
        if use_pivoting:
            pivot(A, b, k)

        for i in range(k+1, n):
            m = round(A[i][k] / A[k][k], 3)
            A[i][k] = 0
            
            for j in range(k+1, n):
                A[i][j] = round(A[i][j] - m * A[k][j], 3)
                
            b[i] = round(b[i] - m * b[k], 3)

    return back_substitution(A, b)


In [128]:
def LU_decomposition(A, b, use_pivoting=False):
    A = A.astype(np.float64)
    b = b.astype(np.float64)

    m, n = A.shape
    
    if m != n:
        raise ValueError("Matrix must be square")
    
    Pivot = np.arange(1, m + 1)
    PdU = 1.0
    Info = 0
    LU = A.copy()

    for j in range(n):
        if use_pivoting:
            pivot(LU, b, j)
        
        Pivot[j], Pivot[np.argmax(np.abs(LU[j:, j])) + j] = Pivot[np.argmax(np.abs(LU[j:, j])) + j], Pivot[j]

        if LU[j, j] == 0:
            if Info == 0:
                Info = j + 1
            continue

        PdU *= LU[j, j]

        if abs(LU[j, j]) != 0:
            r = 1 / LU[j, j]
            for i in range(j + 1, m):
                Mult = LU[i, j] * r
                LU[i, j] = Mult
                LU[i, j + 1:] -= Mult * LU[j, j + 1:]
        else:
            if Info == 0:
                Info = j + 1

    if Info != 0:
        raise ValueError(f"Matrix is singular at row {Info}")

    y = forward_substitution_LU(LU, b, Pivot)

    x = back_substitution(LU, y)

    return x, LU, Pivot, PdU, Info


In [129]:
def cholesky_decomposition(A, b, use_pivoting=False):
    n = A.shape[0]
    L = np.zeros_like(A)
    Det = 1.0
    Info = 0
    
    if use_pivoting:
        for k in range(n):
            pivot(A, b, k)
    
    for j in range(n):
        Soma = 0.0
        for k in range(j):
            Soma += L[j, k] * L[j, k]
        
        t = A[j, j] - Soma
        if t > 0:
            L[j, j] = np.sqrt(t)
            r = 1 / L[j, j]
            Det *= t
        else:
            Info = j + 1
            raise ValueError("Matrix is not positive definite")
        
        for i in range(j + 1, n):
            Soma = 0.0
            for k in range(j):
                Soma += L[i, k] * L[j, k]
            L[i, j] = (A[i, j] - Soma) * r
    
    y = forward_substitution(L, b)
    x = back_substitution(L.T, y)
    
    return x, L, Det, Info


In [130]:
def gauss_jacobi(A, b, toler, iter_max):
    n = len(b)
    x = np.zeros_like(b)
    v = np.zeros_like(b)
    iter_count = 0
    
    for i in range(n):
        x[i] = b[i] / A[i, i]
    
    while True:
        iter_count += 1
        
        for i in range(n):
            sum = 0
            for j in range(n):
                if i != j:
                    sum += A[i, j] * x[j]
            v[i] = (b[i] - sum) / A[i, i]
        
        norma_num = 0
        norma_den = 0
        
        for i in range(n):
            t = abs(v[i] - x[i])
            if t > norma_num:
                norma_num = t
            if abs(v[i]) > norma_den:
                norma_den = abs(v[i])
            x[i] = v[i]
        
        norma_rel = norma_num / norma_den
        # print(f"Iteração: {iter_count}, x: {x}, NormaRel: {norma_rel}")
        
        if norma_rel <= toler or iter_count >= iter_max:
            break
    
    if norma_rel <= toler:
        info = 0
    else:
        info = 1
    
    return x, iter_count, info


In [131]:
def gauss_seidel(A, b, tol, iter_max):
    n = len(b)
    x = np.zeros_like(b, dtype=np.double)
    iter_count = 0
    info = 1

    for i in range(n):
        x[i] = b[i] / A[i, i]

    for iter_count in range(1, iter_max + 1):
        # x_old = np.copy(x)
        norma_num = 0
        norma_den = 0

        for i in range(n):
            soma = 0
            
            for j in range(n):
                if i != j:
                    soma += A[i, j] * x[j]
                    
            v = x[i]
            x[i] = (b[i] - soma) / A[i, i]
            t = abs(v - x[i])
            
            if t > norma_num:
                norma_num = t
                
            if abs(x[i]) > norma_den:
                norma_den = abs(x[i])

        norma_rel = norma_num / norma_den
        # print(f"Iteração {iter_count}: x = {x}, NormaRel = {norma_rel}")

        if norma_rel <= tol or iter_count >= iter_max:
            if norma_rel <= tol:
                info = 0
            break

    return x, iter_count, info


In [ ]:
def newton():
    

# Exercícios

## Exercício 1

In [132]:
A1 = np.array([
    [0, 3, 2],
    [1, 4, 1],
    [0, 2, 5]])

b1 = np.array([5, 6, 7])
# 0x+3y+2z=5, 1x+4y+1z=6, 0x+2y+5z=7
# 0x+3y+2z=5
# 1x+4y+1z=6
# 0x+2y+5z=7

A2 = np.array([
    [-2, -2, 0],
    [1, 3, -1],
    [0, -1, 2]])

b2 = np.array([-1, 3, 1])
# -2x+-2y+0z=-1
# 1x+3y+-1z=3
# 0x-1y+2z=1

A3 = np.array([
    [1, 2, 3],
    [2, 6, 0],
    [1, 0, 4]])

b3 = np.array([4, 8, 5])

# -----------------------------------------------------------------------

# print(gauss_elimination(A1, b1, use_pivoting=False))
# print(gauss_elimination(A2, b2, use_pivoting=False))
# print(gauss_elimination(A3, b3, use_pivoting=False))

# sem pivotamento
# [nan nan nan]
# [-1.5  2.   1.5]
# [ 5.8 -0.6 -0.2]

# com pivotamento
# [1. 1. 1.]
# [-1.5  2.   1.5]
# [ 5.8 -0.6 -0.2]

# -----------------------------------------------------------------------

# solution, LU, Pivot, PdU, Info = LU_decomposition(A1, b1, True)
# print("Solution:", solution)
# print("LU matrix:\n", LU)
# print("Pivot indices:", Pivot)
# print("Product of the diagonal of U:", PdU)
# print("Info:", Info)

# solution, LU, Pivot, PdU, Info = LU_decomposition(A2, b2, True)
# print("Solution:", solution)
 
# solution, LU, Pivot, PdU, Info = LU_decomposition(A3, b3, True)
# print("Solution:", solution)


# sem pivotamento
# ???
# Solution: [-1.5  2.   1.5]
# Solution: [ 5.8 -0.6 -0.2]

# com pivotamento
# [1. 1. 1.]
# [-1.5  2.   1.5]
# [ 5.8 -0.6 -0.2]

# -----------------------------------------------------------------------

x, L, Det, Info = cholesky_decomposition(A1, b1, True)
print(x)
# print("Lower triangular matrix L:\n", L)
# print("Determinant:", Det)
# print("Info:", Info)

# matrix is not symmetric matrix, so matrix is not positive definite

# x, L, Det, Info = cholesky_decomposition(A2, b2, True)
# print(x)

x, L, Det, Info = cholesky_decomposition(A3, b3, True)
print(x)
    
# sem pivotamento


# com pivotamento



[ 6. 11. -3.]
[ 26. -11.  -7.]


## Exercício 2

In [133]:
A1 = np.array([
    [1, 1, 3],
    [1, 1, 4],
    [5, 2, 1]])

b1 = np.array([-2, -3, 4])

# 1x+1y+3z=-2, ax+1y+4z=-3, 5x+2y+1z=4
# 1x+1y+3z=-2
# ax+1y+4z=-3
# 5x+2y+1z=4

print(gauss_elimination(A1, b1, use_pivoting=True))


[ 1.  0. -1.]


## Exercício 3

In [134]:
# 17x – 2y – 3z = 500
# –5x + 21y – 2z = 200
# –5x – 5y + 22z = 30

A1 = np.array([
    [17, -2, -3],
    [-5, 21, -2],
    [-5, -5, 22]])

b1 = np.array([500, 200, 30])

x0 = np.array([34, 19, 13], dtype=float)

epsilon = 1e-10
max_iterations = 10

x, iter_count, info = gauss_jacobi(A1, b1, epsilon, max_iterations)
print(f"Solução: {x}")

x, iter_count, info = gauss_seidel(A1, b1, epsilon, max_iterations)
print(f"Solução: {x}")

# Solução: [33 18 12]
# Solução: [33.99631415 18.89282676 13.38389566]

Solução: [33 18 12]
Solução: [33.99631413 18.89282676 13.38389566]


## Exercício 4

In [135]:
# 2x + 5y = -3
# 3x + y = 2

A1 = np.array([
    [2, 5,],
    [3, 1]])

b1 = np.array([-3, 2])

toler = 1e-10
iter_max = 100

x, iter_count, info = gauss_jacobi(A1, b1, toler, iter_max)
print(f"Solução: {x}")
# print(f"Número de iterações: {iter_count}")
# print(f"Info: {info}")

x, iter_count, info = gauss_seidel(A1, b1, toler, iter_max)
print(f"Solução: {x}")

Solução: [   48941378 -1640500820]
Solução: [-3.20720219e+87  9.62160656e+87]


C:\Users\thiag\AppData\Local\Temp\ipykernel_14120\1094892365.py:17: RuntimeWarning: overflow encountered in scalar multiply
  sum += A[i, j] * x[j]
C:\Users\thiag\AppData\Local\Temp\ipykernel_14120\1094892365.py:24: RuntimeWarning: overflow encountered in scalar subtract
  t = abs(v[i] - x[i])


## Exercício 5

In [136]:
# 3sen(x) – 4y –12z – 1 = 0
# 4x^2 – 8y – 10z + 5 = 0
# 2e^x + 2y + 3z – 8 = 0

# aproximação inicial com o valor (0 ,0 ,0)

# newton()